In [1]:
# library(devtools)
# document()
# setwd("../")
# install("itecv")
# library(itecv)
# setwd("itecv")

In [2]:
library(foreach)
library(doParallel)
library(Matching)
library(plyr)
library(tidyverse)
library(stringr)
library(magrittr)
library(caret)
library(itecv)
# detach("package:plyr", unload=TRUE)

Loading required package: iterators
Loading required package: parallel
Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ dplyr::arrange()    masks plyr::arrange()
✖ purrr::compact()    masks plyr::compact()
✖ dplyr::count()      masks plyr::count()
✖ dplyr::failwith()   masks plyr::failwi

In [3]:
loop_test = function(setup) {
    DGP = setup$DGP
    n_train = setup$n_train
    n_test = setup$n_test
    n_folds = setup$n_folds
    rep = setup$rep
    models = setup$models
    
    datas = setup_data(DGP$DGP, n_train, n_test, n_folds)
    estimates = datas %$% get_estimates(data, models, cv_index, test_index) 
    errors =  estimates %$% get_errors(cv_estimates, test_estimates, datas$aux_data) 
    true_selection_error = errors$true_selection_error %>%
        mutate(DGP=DGP$name, n_train=n_train, n_test=n_test, n_folds=n_folds, rep=rep)
    
    return(true_selection_error)
}

In [4]:
models = list(
# ranger_spec = list(method = "ranger",
#                    tune_grid = expand.grid(mtry = c(3,5),
#                                            splitrule="variance",
#                                            min.node.size=5)),
gbm_spec = list(method = "gbm",
                tune_grid = expand.grid(n.trees = seq(1,501,20), 
                                        interaction.depth=3, 
                                        shrinkage = 0.2, 
                                        n.minobsinnode=3)),
enet_spec = list(method = "enet",
                 tune_grid = expand.grid(fraction=0.5,
                                         lambda=exp(seq(-5,2,0.2))))
# nnet_spec = list(method = "nnet",
#                  tune_grid = expand.grid(size = seq(10,30,5),
#                                          decay = exp(seq(-7,0,1))))
)

simulation_params = list(DGP = powers_DGPs() %>% 
                             imap(~list(name=.y, DGP=.x)),
                         n_test = 2000,
                         n_train = 1000,
                         n_folds = c(2,5,10),
                         rep = 26:50) %>%
cross
setups = simulation_params %>%
    map(~c(.,"models"=list(models)))

In [5]:
# derp = setups[[1]] %>% loop_test %>% data.frame

In [6]:
# results = setups %>% 
#     map(~loop_test(.)) %>%
#     bind_rows

In [7]:
cl = makeCluster(8)
registerDoParallel(cl)
 
results = foreach(setup = setups, 
        .combine = bind_rows,
        .packages = (.packages()))  %dopar%  
loop_test(setup) 

write_csv(results, "data/experimental_results_2.csv")

In [8]:
stopCluster(cl)